In [1]:
# Examples of Using Big Query as backend to Query data.
# Examples assume 
# a. google auth login - commands have been run to set up auth and default project
# b. the default project has `synthea_big` dataset with Patient, Encounter and Observation tables with sample data
# that is used in this notebook

In [2]:
import warnings
from query_lib_big_query import _BigQueryPatientQuery

_BIGQUERY_DATASET = "synthea_big"
_CODE_SYSTEM = "http://www.ampathkenya.org"
_PROJECT_NAME = "fhir-analytics-test"
warnings.filterwarnings("ignore")

# Example 1: Fetch Encounter data

In [3]:
pq = _BigQueryPatientQuery(
        bq_dataset=_BIGQUERY_DATASET, code_system="http://www.ampathkenya.org", project_name=_PROJECT_NAME
    )
pq.encounter_constraints(
        typeSystem="http://fhir.openmrs.org/code-system/encounter-type"
    )

encounter_df = pq.get_patient_encounter_view(
        force_location_type_columns=False
    )
encounter_df.head(5)

,encPatientId,locationId,encTypeSystem,encTypeCode,locationDisplay,num_encounters,firstDate,lastDate
0,fc387977-6f2e-4ee0-8bfa-767e7fadfd49,2131aff8-2e2a-480a-b7ab-4ac53250262b,http://fhir.openmrs.org/code-system/encounter-...,5021b1a1-e7f6-44b4-ba02-da2f2bcf8718,Isolation Ward,16,1944-01-31T05:29:31+00:00,None
1,14f70276-b8bb-40db-adaa-603a68a1f7a6,2131aff8-2e2a-480a-b7ab-4ac53250262b,http://fhir.openmrs.org/code-system/encounter-...,5021b1a1-e7f6-44b4-ba02-da2f2bcf8718,Isolation Ward,16,1941-04-23T08:29:15+00:00,None
2,50b579ef-aeb6-4fe4-a4b1-0397aea91c49,2131aff8-2e2a-480a-b7ab-4ac53250262b,http://fhir.openmrs.org/code-system/encounter-...,5021b1a1-e7f6-44b4-ba02-da2f2bcf8718,Isolation Ward,18,1914-05-02T02:54:26+00:00,None
3,b6103345-e809-401a-bcbb-914313d89fdc,2131aff8-2e2a-480a-b7ab-4ac53250262b,http://fhir.openmrs.org/code-system/encounter-...,5021b1a1-e7f6-44b4-ba02-da2f2bcf8718,Isolation Ward,16,1912-06-20T00:23:18+00:00,None
4,09890e2f-1c2e-44d7-8d2f-a2089700ed44,2131aff8-2e2a-480a-b7ab-4ac53250262b,http://fhir.openmrs.org/code-system/encounter-...,5021b1a1-e7f6-44b4-ba02-da2f2bcf8718,Isolation Ward,24,1949-01-13T22:13:16+00:00,None


# Example 2: Encounter Data based on codes

In [4]:
pq = _BigQueryPatientQuery(
        bq_dataset=_BIGQUERY_DATASET, code_system="dummy_code_system", project_name=_PROJECT_NAME
    )
test_codes = [
        "e22e39fd-7db2-45e7-80f1-60fa0d5a4378",
        "181820aa-88c9-479b-9077-af92f5364329",
    ]

pq.encounter_constraints(typeCode=test_codes)

encounter_df = pq.get_patient_encounter_view(
        force_location_type_columns=False
    )
encounter_df.head(5)

,encPatientId,locationId,encTypeSystem,encTypeCode,locationDisplay,num_encounters,firstDate,lastDate
0,8295eb5b-fba6-4e83-a5cb-2817b135cd27,b1a8b05e-3542-4037-bbd3-998ee9c40574,http://fhir.openmrs.org/code-system/encounter-...,e22e39fd-7db2-45e7-80f1-60fa0d5a4378,Inpatient Ward,2,2019-02-28T10:43:00+00:00,None
1,8295eb5b-fba6-4e83-a5cb-2817b135cd27,b1a8b05e-3542-4037-bbd3-998ee9c40574,http://fhir.openmrs.org/code-system/encounter-...,181820aa-88c9-479b-9077-af92f5364329,Inpatient Ward,2,2019-03-03T10:43:00+00:00,None


# Example 3: Fetch Encounter data based on location id

In [5]:
pq = _BigQueryPatientQuery(
        bq_dataset=_BIGQUERY_DATASET, code_system="dummy_code_system", project_name=_PROJECT_NAME
    )

test_locations = ["2131aff8-2e2a-480a-b7ab-4ac53250262b"]

pq.encounter_constraints(locationId=test_locations)

encounter_df = pq.get_patient_encounter_view(
        force_location_type_columns=False
    )
encounter_df.head(5)

,encPatientId,locationId,encTypeSystem,encTypeCode,locationDisplay,num_encounters,firstDate,lastDate
0,fc387977-6f2e-4ee0-8bfa-767e7fadfd49,2131aff8-2e2a-480a-b7ab-4ac53250262b,http://fhir.openmrs.org/code-system/encounter-...,5021b1a1-e7f6-44b4-ba02-da2f2bcf8718,Isolation Ward,16,1944-01-31T05:29:31+00:00,None
1,14f70276-b8bb-40db-adaa-603a68a1f7a6,2131aff8-2e2a-480a-b7ab-4ac53250262b,http://fhir.openmrs.org/code-system/encounter-...,5021b1a1-e7f6-44b4-ba02-da2f2bcf8718,Isolation Ward,16,1941-04-23T08:29:15+00:00,None
2,50b579ef-aeb6-4fe4-a4b1-0397aea91c49,2131aff8-2e2a-480a-b7ab-4ac53250262b,http://fhir.openmrs.org/code-system/encounter-...,5021b1a1-e7f6-44b4-ba02-da2f2bcf8718,Isolation Ward,18,1914-05-02T02:54:26+00:00,None
3,b6103345-e809-401a-bcbb-914313d89fdc,2131aff8-2e2a-480a-b7ab-4ac53250262b,http://fhir.openmrs.org/code-system/encounter-...,5021b1a1-e7f6-44b4-ba02-da2f2bcf8718,Isolation Ward,16,1912-06-20T00:23:18+00:00,None
4,09890e2f-1c2e-44d7-8d2f-a2089700ed44,2131aff8-2e2a-480a-b7ab-4ac53250262b,http://fhir.openmrs.org/code-system/encounter-...,5021b1a1-e7f6-44b4-ba02-da2f2bcf8718,Isolation Ward,24,1949-01-13T22:13:16+00:00,None


# Example 4: Encounter data with multiple condition

In [6]:
test_locations = ["b1a8b05e-3542-4037-bbd3-998ee9c40574"]
test_codes = [
    "e22e39fd-7db2-45e7-80f1-60fa0d5a4378",
    "181820aa-88c9-479b-9077-af92f5364329",
]
test_type_system = "http://fhir.openmrs.org/code-system/encounter-type"

pq = _BigQueryPatientQuery(
    bq_dataset=_BIGQUERY_DATASET, code_system="http://www.ampathkenya.org", project_name=_PROJECT_NAME
)
pq.encounter_constraints(
    typeSystem=test_type_system,
    locationId=test_locations,
    typeCode=test_codes
)
encounter_df = pq.get_patient_encounter_view(
    force_location_type_columns=False,
    sample_count=10,
)
encounter_df.head(5)

,encPatientId,locationId,encTypeSystem,encTypeCode,locationDisplay,num_encounters,firstDate,lastDate
0,8295eb5b-fba6-4e83-a5cb-2817b135cd27,b1a8b05e-3542-4037-bbd3-998ee9c40574,http://fhir.openmrs.org/code-system/encounter-...,e22e39fd-7db2-45e7-80f1-60fa0d5a4378,Inpatient Ward,2,2019-02-28T10:43:00+00:00,None
1,8295eb5b-fba6-4e83-a5cb-2817b135cd27,b1a8b05e-3542-4037-bbd3-998ee9c40574,http://fhir.openmrs.org/code-system/encounter-...,181820aa-88c9-479b-9077-af92f5364329,Inpatient Ward,2,2019-03-03T10:43:00+00:00,None


# Example 5: Observation Data with multiple conditions

In [7]:
_VL_CODE = "856"  # HIV VIRAL LOAD
_ARV_PLAN = "1255"  # ANTIRETROVIRAL PLAN
end_date = "2017-01-01"
start_date = "2000-01-01"
_BASE_URL = ""

# Creating a new `patient_query` to drop all previous constraints
# and recreate flat views.
patient_query = _BigQueryPatientQuery(
    bq_dataset=_BIGQUERY_DATASET, code_system=_CODE_SYSTEM, project_name=_PROJECT_NAME
)

patient_query.include_obs_values_in_time_range(
    _VL_CODE, min_time=start_date, max_time=end_date
)
patient_query.include_obs_values_in_time_range(
    _ARV_PLAN, min_time=start_date, max_time=end_date
)

other_end_date = "2020-01-01"
other_start_date = "1998-01-01"
patient_query.include_all_other_codes(
    min_time=other_start_date, max_time=other_end_date
)

patient_query.encounter_constraints(
    #locationId=["7f65d926-57d6-4402-ae10-a5b3bcbf7986"]
)

obs_df = patient_query.get_patient_obs_view()
obs_df.head()
#obs_df[obs_df['patientId'=='00c1426f-ca04-414a-8db7-043bb41b64d2']]

,patientId,code,birthDate,gender,num_obs,min_value,max_value,min_date,max_date,last_value,first_value,last_value_code,first_value_code
0,15c66194-59d1-4a28-bc35-ad4aa140c261,844,1964-06-27,male,28,NaN,NaN,1999-07-12T13:00:12+00:00,2018-03-28T13:00:12+00:00,None,None,1304AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA,1304
1,15c66194-59d1-4a28-bc35-ad4aa140c261,1030,1964-06-27,male,28,NaN,NaN,1999-07-12T13:00:12+00:00,2018-03-28T13:00:12+00:00,None,None,1304AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA,1304
2,9e2fe61b-a55d-4c2d-9b5c-7f157c84ddfe,844,1981-11-05,female,10,NaN,NaN,2005-05-31T04:58:49+00:00,2009-04-17T04:58:49+00:00,None,None,704,664
3,9e2fe61b-a55d-4c2d-9b5c-7f157c84ddfe,1030,1981-11-05,female,10,NaN,NaN,2005-05-31T04:58:49+00:00,2009-04-17T04:58:49+00:00,None,None,704,664
4,1b961cdf-0d7b-4dc0-aa35-72d504c02941,844,1955-04-11,female,13,NaN,NaN,2001-10-02T16:10:01+00:00,2011-01-23T16:10:01+00:00,None,None,1138AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA,1138


In [8]:
obs_df[obs_df['patientId']=='00c1426f-ca04-414a-8db7-043bb41b64d2']

,patientId,code,birthDate,gender,num_obs,min_value,max_value,min_date,max_date,last_value,first_value,last_value_code,first_value_code
1110,00c1426f-ca04-414a-8db7-043bb41b64d2,844,1994-04-18,male,6,NaN,NaN,2009-07-07T01:44:23+00:00,2012-07-02T01:44:23+00:00,None,None,1138AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA,1138
1111,00c1426f-ca04-414a-8db7-043bb41b64d2,1030,1994-04-18,male,6,NaN,NaN,2009-07-07T01:44:23+00:00,2012-07-02T01:44:23+00:00,None,None,1138AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA,1138
32667,00c1426f-ca04-414a-8db7-043bb41b64d2,1271,1994-04-18,male,100,NaN,NaN,2009-07-07T01:44:23+00:00,2012-07-02T01:44:23+00:00,None,None,856AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA,1107
38177,00c1426f-ca04-414a-8db7-043bb41b64d2,6174,1994-04-18,male,19,NaN,NaN,2009-07-07T01:44:23+00:00,2012-07-02T01:44:23+00:00,None,None,967,140238AAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
44264,00c1426f-ca04-414a-8db7-043bb41b64d2,5085,1994-04-18,male,6,101.0,244.0,2009-07-07T01:44:23+00:00,2012-07-02T01:44:23+00:00,244,101,None,None
48777,00c1426f-ca04-414a-8db7-043bb41b64d2,5087,1994-04-18,male,3,50.0,111.0,2009-07-07T01:44:23+00:00,2012-07-02T01:44:23+00:00,65,111,None,None
54068,00c1426f-ca04-414a-8db7-043bb41b64d2,5092,1994-04-18,male,3,73.0,90.0,2009-07-07T01:44:23+00:00,2012-07-02T01:44:23+00:00,75,73,None,None
55506,00c1426f-ca04-414a-8db7-043bb41b64d2,856,1994-04-18,male,3,182520.0,836370.0,2009-07-07T01:44:23+00:00,2012-07-02T01:44:23+00:00,836370,823690,None,None
71809,00c1426f-ca04-414a-8db7-043bb41b64d2,5088,1994-04-18,male,3,36.0,38.0,2009-07-07T01:44:23+00:00,2012-07-02T01:44:23+00:00,36,38,None,None
77702,00c1426f-ca04-414a-8db7-043bb41b64d2,5089,1994-04-18,male,1,64.0,64.0,2009-07-07T01:44:23+00:00,2009-07-07T01:44:23+00:00,64,64,None,None


In [9]:
patient_id = "01188340-b29f-471a-9cc8-2b2247aa9f84"
obs_df[obs_df['patientId']==patient_id]

,patientId,code,birthDate,gender,num_obs,min_value,max_value,min_date,max_date,last_value,first_value,last_value_code,first_value_code
6888,01188340-b29f-471a-9cc8-2b2247aa9f84,1088,1953-08-02,male,133,NaN,NaN,1998-02-22T11:44:13+00:00,2006-09-04T11:44:13+00:00,None,None,84795AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA,104567AAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
13458,01188340-b29f-471a-9cc8-2b2247aa9f84,1111,1953-08-02,male,140,NaN,NaN,1998-02-22T11:44:13+00:00,2006-09-04T11:44:13+00:00,None,None,78280AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA,1107
15890,01188340-b29f-471a-9cc8-2b2247aa9f84,1250,1953-08-02,male,135,NaN,NaN,1998-02-22T11:44:13+00:00,2006-09-04T11:44:13+00:00,None,None,84795AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA,628
22319,01188340-b29f-471a-9cc8-2b2247aa9f84,1261,1953-08-02,male,16,NaN,NaN,1998-02-22T11:44:13+00:00,2006-09-04T11:44:13+00:00,None,None,1260AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA,1256
25231,01188340-b29f-471a-9cc8-2b2247aa9f84,1265,1953-08-02,male,16,NaN,NaN,1998-02-22T11:44:13+00:00,2006-09-04T11:44:13+00:00,None,None,1107AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA,981
29728,01188340-b29f-471a-9cc8-2b2247aa9f84,1270,1953-08-02,male,159,NaN,NaN,1998-02-22T11:44:13+00:00,2006-09-04T11:44:13+00:00,None,None,82900AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA,1108
35398,01188340-b29f-471a-9cc8-2b2247aa9f84,1271,1953-08-02,male,261,NaN,NaN,1998-02-22T11:44:13+00:00,2006-08-05T11:44:13+00:00,None,None,856AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA,1107
39135,01188340-b29f-471a-9cc8-2b2247aa9f84,6174,1953-08-02,male,33,NaN,NaN,1998-02-22T11:44:13+00:00,2006-08-05T11:44:13+00:00,None,None,6171,133027AAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
45254,01188340-b29f-471a-9cc8-2b2247aa9f84,5085,1953-08-02,male,12,56.0,227.0,1998-02-22T11:44:13+00:00,2006-08-05T11:44:13+00:00,62,181,None,None
48706,01188340-b29f-471a-9cc8-2b2247aa9f84,5087,1953-08-02,male,6,50.0,197.0,1998-02-22T11:44:13+00:00,2006-08-05T11:44:13+00:00,104,178,None,None


In [10]:
patient_id = '01b747b4-2029-4842-bf32-e3157c4f4f06'
patient_id = '04472fb9-14d2-4510-8459-db44c9271986'
obs_df[obs_df['patientId']==patient_id] #[['code', 'first_value_code', 'last_value_code']]

,patientId,code,birthDate,gender,num_obs,min_value,max_value,min_date,max_date,last_value,first_value,last_value_code,first_value_code
366,04472fb9-14d2-4510-8459-db44c9271986,844,1974-07-08,male,6,NaN,NaN,2011-09-30T20:26:46+00:00,2014-03-05T20:26:46+00:00,None,None,1138AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA,1138
367,04472fb9-14d2-4510-8459-db44c9271986,1030,1974-07-08,male,6,NaN,NaN,2011-09-30T20:26:46+00:00,2014-03-05T20:26:46+00:00,None,None,1138AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA,1138
6516,04472fb9-14d2-4510-8459-db44c9271986,1088,1974-07-08,male,52,NaN,NaN,2012-10-20T20:26:46+00:00,2014-03-05T20:26:46+00:00,None,None,84795AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA,104565AAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
13791,04472fb9-14d2-4510-8459-db44c9271986,1111,1974-07-08,male,33,NaN,NaN,2012-10-20T20:26:46+00:00,2014-03-05T20:26:46+00:00,None,None,75948AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA,1107
17950,04472fb9-14d2-4510-8459-db44c9271986,1250,1974-07-08,male,34,NaN,NaN,2012-10-20T20:26:46+00:00,2014-03-05T20:26:46+00:00,None,None,86663AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA,623
26206,04472fb9-14d2-4510-8459-db44c9271986,1268,1974-07-08,male,6,NaN,NaN,2012-10-20T20:26:46+00:00,2014-03-05T20:26:46+00:00,None,None,1257AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA,1259
31486,04472fb9-14d2-4510-8459-db44c9271986,1270,1974-07-08,male,61,NaN,NaN,2012-10-20T20:26:46+00:00,2014-03-05T20:26:46+00:00,None,None,82900AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA,1108
34896,04472fb9-14d2-4510-8459-db44c9271986,1271,1974-07-08,male,110,NaN,NaN,2011-09-30T20:26:46+00:00,2014-03-05T20:26:46+00:00,None,None,856AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA,1042
39920,04472fb9-14d2-4510-8459-db44c9271986,6174,1974-07-08,male,26,NaN,NaN,2011-09-30T20:26:46+00:00,2014-03-05T20:26:46+00:00,None,None,967,138905AAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
43899,04472fb9-14d2-4510-8459-db44c9271986,2154,1974-07-08,male,5,NaN,NaN,2012-10-20T20:26:46+00:00,2014-03-05T20:26:46+00:00,None,None,817AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA,1652AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
